# Data cleaning - Features

In [385]:
import pandas as pd
import numpy as np
import os

In [386]:
os.chdir("C:/Users/Kenia/OneDrive - The University of Chicago/03_Q3/03_Machine_Learning/05_Project/Data_KGN")

In [387]:
# Import datasets
df_crime = pd.read_csv('Crime.csv')
df_ed = pd.read_csv('Education.csv')
df_racial = pd.read_csv('Racial_transition.csv')
df_housing = pd.read_csv('Housing.csv')

# Build a dataframe with all variables
dfs = [df_crime, df_ed, df_racial, df_housing]
df = pd.concat(dfs, axis=1).reindex(dfs[0].index)

In [388]:
df = df.drop([0, 1 , 2, 3]) # Drop description of variables
df = df.loc[:,~df.columns.duplicated()] # Drop duplicate columns
df.head()

,Layer,Name,GEOID,CZP_2016-2020,CZP_2012-2016,CZP_2011-2015,CZP_2010-2014,CZP_2009-2013,CZP_2008-2012,CZP_2007-2011,...,VAC_2010-2014,VAC_2009-2013,VAC_2008-2012,VAC_2007-2011,RBS_2016-2020,RBS_2015-2019,RBS_2014-2018,RBS_2013-2017,RBS_2012-2016,RBS_2011-2015
4,Zip code,60601,60601,1609.84212,6645.948,6263.8484,6175.5184,6281.8128,6471.6064,6509.8118,...,21.83583606,22.87755844,19.72918249,25.99277978,18.8761593,23.64490686,20.18448182,21.67218866,19.31865393,17.15490457
5,Zip code,60602,60602,197.59506,824.9727,782.5987,775.3808,789.0716,809.6934,813.7886,...,14.4549763,15.35756155,16.18421053,18.30161054,10.55555556,4.927536232,30.23872679,27.56264237,31.2195122,42.64069264
6,Zip code,60603,60603,246.87882,1074.7829,1066.0261,1084.804,1105.7748,1122.7882,1144.3376,...,43.06666667,40.58441558,38.87587822,37.10691824,12.16931217,16.52542373,19.66527197,18.70229008,18.22222222,13.61867704
7,Zip code,60604,60604,260.08338,1145.849,1117.8522,1131.1358,1153.282,1162.7758,1162.9329,...,41.26637555,34.42211055,32.47663551,31.53638814,42.8125,19.94382022,15.21035599,13.41463415,14.22413793,19.17098446
8,Zip code,60605,60605,2350.67174,10839.2012,10795.4496,11010.9108,11280.003,11633.9454,11856.5887,...,9.427994616,8.687870026,10.2194539,10.19604756,18.30438164,19.77475118,20.85369827,20.73288332,25.35766423,25.73438146


In [389]:
# Reshape data to create year variable
df = pd.wide_to_long(df, stubnames = ["CZP", "CZV", "CZH", "EDE", "EDB", "PCT-W", "RBU", "VAC", "RBS"], 
                i = ["Layer", "Name", "GEOID"], 
                j = "year", 
                sep = "_", 
                suffix=".+").reset_index()

In [390]:
df.head()

,Layer,Name,GEOID,year,CZP,CZV,CZH,EDE,EDB,PCT-W,RBU,VAC,RBS
0,Zip code,60601,60601,2001-2005,8412.7806,385.1064,1.7794,NaN,NaN,NaN,NaN,NaN,NaN
1,Zip code,60601,60601,2002-2006,7445.9046,397.8528,1.7794,NaN,NaN,NaN,NaN,NaN,NaN
2,Zip code,60601,60601,2003-2007,7259.1014,419.313,1.1982,NaN,NaN,NaN,NaN,NaN,NaN
3,Zip code,60601,60601,2004-2008,7234.331,433.226,1.4888,NaN,NaN,NaN,NaN,NaN,NaN
4,Zip code,60601,60601,2005-2009,6900.7998,461.0436,1.1982,NaN,NaN,NaN,NaN,NaN,NaN


In [391]:
# Keep just relevant years
df = df[df.year.isin(['2008-2012', '2012-2016', '2016-2020'])]
df.head()

,Layer,Name,GEOID,year,CZP,CZV,CZH,EDE,EDB,PCT-W,RBU,VAC,RBS
7,Zip code,60601,60601,2008-2012,6471.6064,462.962,0.5812,80.7682458386684,98.2458386683739,66.0198141150036,NaN,19.72918249,NaN
11,Zip code,60601,60601,2012-2016,6645.948,435.6814,0.5812,76.821608040201,99.3000717875089,66.3672873311427,36.497715,16.87159847,19.31865393
15,Zip code,60601,60601,2016-2020,1609.84212,140.90324,0.23964,83.4512976070104,94.0680822379508,59.3536829049817,36.77032188,15.92476489,18.8761593
24,Zip code,60602,60602,2008-2012,809.6934,58.9149,0.0902999999999999,83.3910034602076,100.0,61.2708018154312,NaN,16.18421053,NaN
28,Zip code,60602,60602,2012-2016,824.9727,55.6381,0.107,88.249694002448,100.0,72.2044728434505,45.12195122,14.50704225,31.2195122


In [392]:
# Rename variables
df = df.drop(columns=["Layer", "Name", "EDB"]) 
df.rename(columns = {'CZP':'property_crime', 
                     'CZV':'violent_crime',
                     'CZH':'homicide',
                     'EDE':'college',
                     'PCT-W':'non_hip_white',
                     'RBU':'rent_burdened',
                     'VAC':'vacancy',
                     'RBS':'severe_rent_b',
                     'GEOID':'zip'}, 
          inplace = True)

In [393]:
# Replace years to keep upper bound except in the last period (keeping 2019 as reference)
# Note: Rent-burdened variables have no value for the first period
df = df.replace({'year': {'2008-2012': 2012, '2012-2016': 2016, '2016-2020': 2019}})
df.head()

,zip,year,property_crime,violent_crime,homicide,college,non_hip_white,rent_burdened,vacancy,severe_rent_b
7,60601,2012,6471.6064,462.962,0.5812,80.7682458386684,66.0198141150036,NaN,19.72918249,NaN
11,60601,2016,6645.948,435.6814,0.5812,76.821608040201,66.3672873311427,36.497715,16.87159847,19.31865393
15,60601,2019,1609.84212,140.90324,0.23964,83.4512976070104,59.3536829049817,36.77032188,15.92476489,18.8761593
24,60602,2012,809.6934,58.9149,0.0902999999999999,83.3910034602076,61.2708018154312,NaN,16.18421053,NaN
28,60602,2016,824.9727,55.6381,0.107,88.249694002448,72.2044728434505,45.12195122,14.50704225,31.2195122


In [394]:
# Append data --variables on housing, demographics and unemployment
df_half = pd.read_csv('half_features.csv')
df_half = df_half.drop(columns= "percent_over_65unemp_rate")
df_half.rename(columns={'Zip': 'zip'}, inplace = True)

In [395]:
df.dtypes
df_half.dtypes

zip              float64
year               int64
med_rent         float64
perc_under_18    float64
perc_over_65     float64
unemp_rate       float64
eviction_rate    float64
dtype: object

In [396]:
# Make sure both datasets contain all zip codes
print(len(df_half.zip.unique()))
print(len(df.zip.unique()))

58
58


In [397]:
# Change data types to make merge
df['year'] = pd.to_numeric(df.year, downcast="integer")
df_half['year'] = pd.to_numeric(df_half.year, downcast="integer")
df['zip'] = pd.to_numeric(df.zip, downcast="integer")
df_half['zip'] = pd.to_numeric(df_half.zip, downcast="integer")

In [398]:
# Merge with previous dataset
result = pd.merge(df, df_half, on=["zip", "year"])
result

,zip,year,property_crime,violent_crime,homicide,college,non_hip_white,rent_burdened,vacancy,severe_rent_b,med_rent,perc_under_18,perc_over_65,unemp_rate,eviction_rate
0,60601,2012,6471.6064,462.962,0.5812,80.7682458386684,66.0198141150036,NaN,19.72918249,NaN,1726.0,10.202899,7.652174,0.046680,0.519465
1,60601,2016,6645.948,435.6814,0.5812,76.821608040201,66.3672873311427,36.497715,16.87159847,19.31865393,1973.0,24.986945,13.812010,0.053607,0.818005
2,60601,2019,1609.84212,140.90324,0.23964,83.4512976070104,59.3536829049817,36.77032188,15.92476489,18.8761593,2289.0,26.190476,20.595238,0.046621,0.351792
3,60602,2012,809.6934,58.9149,0.0902999999999999,83.3910034602076,61.2708018154312,NaN,16.18421053,NaN,1792.0,5.389408,0.155763,0.027586,0.641695
4,60602,2016,824.9727,55.6381,0.107,88.249694002448,72.2044728434505,45.12195122,14.50704225,31.2195122,2255.0,4.182510,0.494297,0.008000,0.894205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,60707,2016,2307.3989,284.2721,1.3256,26.3945803040317,49.9136958873213,56.2170309,7.233941127,33.0256217,946.0,242.168367,20.433673,0.085644,1.130688
170,60707,2019,308.65954,63.56158,0.83974,27.8434788435123,46.849224678324,56.97119632,7.456087944,27.90245626,1095.0,221.777778,24.814815,0.059983,0.560275
171,60827,2012,1451.4222,899.129,18.1666,12.4467329545455,3.93581378945512,NaN,20.35043608,NaN,819.0,267.555556,11.460317,0.230959,4.460428
172,60827,2016,1372.6342,784.1262,16.0,12.4376956068158,2.66768292682927,43.72193581,20.37893141,30.10144687,726.0,272.163009,15.297806,0.258437,3.363926


In [399]:
# Export data
result.to_csv("features.csv", index=False)